In [1]:
# https://habr.com/ru/post/347564/.
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
#os.environ['THEANO_FLAGS'] = "mode=FAST_RUN,device=cuda:0,floatX=float32"
#import theano
#print(theano.config.device, theano.config.floatX)

import keras
import PIL
import sys
#from google.colab import files # для импорта данных в google colab
#from google.colab import drive # для импорта данных в google colab из google drive
import zipfile # для работы с архивами 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import csv
import tensorflow as tf
import os
import tensorflow as tf

from keras.layers import Input
from keras.applications import xception
#from keras.applications import resnet
from keras.preprocessing import image

from tensorflow.keras.preprocessing import image as im
import time

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from keras.utils import to_categorical
from tensorflow.keras.models import load_model  



Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10513024560051973327
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16181014659712759170
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 32070087476
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3325159402369502349
physical_device_desc: "device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:04:00.0, compute capability: 7.0"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4833630928692382115
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
K.set_learning_phase(1)

In [4]:
class_names = ['1', '2', '3', '4']
print(len(class_names))

4


In [5]:
img_width, img_height = 71, 71
nb_train_samples = 30451
nb_validation_samples = 7434
epochs = 12
batch_size = 8
n_classes = 4

In [6]:
import cv2
im = cv2.imread('home/z_andrei/zoohackathon/i_cropped_train/1/32886.jpg')
print(im)

None


In [7]:
train_data_dir = "/home/z_andrei/zoohackathon/i_cropped_train"    # дальше там папки, а не сами фотографии!!!!
validation_data_dir = "/home/z_andrei/zoohackathon/i_cropped_test"

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    #shear_range=0.2,
    zoom_range=0.2,
    #fill_mode = 'constant',
    #cval = 1,
    rotation_range = 5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip = False,
    vertical_flip = False)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')


Found 30451 images belonging to 4 classes.
Found 7434 images belonging to 4 classes.


In [9]:
def build_model_Xception():
  base_model = xception.Xception(include_top=True, 
                                      weights = None,
                                      input_tensor = None,
                                      input_shape=(img_width, img_height, 3),
                                      pooling=None,
                                      classes = 4)
  
  for layer in base_model.layers:
    layer.trainable = True
      
  model = Model(inputs = base_model.input, outputs = base_model.output)
  return model

In [10]:
model = build_model_Xception()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc', 'mse'])

In [11]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4) # остановка обучения, если loss на валидационном множесте улучшается менее чем на 10^-4
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
filepath="weights-zoo-128-{epoch:02d}-{val_acc:.2f}.hdf5"
check = ModelCheckpoint(filepath, monitor = "val_acc", save_best_only = False) # сохранение лучшей (с наибольшим acc на валидационном множестве) сети
callbacks_list = [early_stop, reduce_lr , check]

In [12]:
model_history = model.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = callbacks_list,
    steps_per_epoch = nb_train_samples // batch_size )

Epoch 1/12
3806/3806 [==============================] - 367s 96ms/step - loss: 0.4795 - acc: 0.8372 - mse: 0.0589 - val_loss: 0.1898 - val_acc: 0.8924 - val_mse: 0.0395
Epoch 2/12
3806/3806 [==============================] - 362s 95ms/step - loss: 0.3381 - acc: 0.8842 - mse: 0.0424 - val_loss: 0.0204 - val_acc: 0.9185 - val_mse: 0.0301
Epoch 3/12
3806/3806 [==============================] - 341s 90ms/step - loss: 0.2770 - acc: 0.9030 - mse: 0.0356 - val_loss: 0.0783 - val_acc: 0.9333 - val_mse: 0.0252
Epoch 4/12
3806/3806 [==============================] - 349s 92ms/step - loss: 0.2345 - acc: 0.9174 - mse: 0.0305 - val_loss: 0.0933 - val_acc: 0.9185 - val_mse: 0.0306
Epoch 5/12
3806/3806 [==============================] - 361s 95ms/step - loss: 0.2098 - acc: 0.9272 - mse: 0.0273 - val_loss: 0.0166 - val_acc: 0.9362 - val_mse: 0.0231
Epoch 6/12
3806/3806 [==============================] - 359s 94ms/step - loss: 0.1864 - acc: 0.9357 - mse: 0.0243 - val_loss: 0.1380 - val_acc: 0.9519 - va